# 1. 모듈 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/단위프로젝트_1조/EDA

/content/drive/MyDrive/단위프로젝트_1조/EDA


In [3]:
pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 84.3 MB/s eta 0:00:00


In [4]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 56.8 MB/s eta 0:00:00


In [5]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!apt install fonts-nanum

!sudo apt install font-manager
!rm -rf ~/.cache/matplotlib
!pip install koreanize-matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import folium
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from datetime import datetime
import matplotlib.dates as md
from matplotlib import rc
import koreanize_matplotlib
from datetime import datetime
import matplotlib.dates as md
from pyproj import Transformer
import geopandas as gpd
import os

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 6s (1,550 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 122519 files and di

#2. 데이터 불러오기

In [14]:
df1 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/직장인구(공통데이터).csv')
df2 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/상주인구(공통데이터).csv')
df3 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/생활인구(공통데이터).csv')

## 2.1. 구별로 저장(영등포구만)

In [15]:
sgg = '영등포구'                     # sgg 에 구명 대입 
dfs = [df1, df2, df3]   # 각 데이터 프레임들 넣고
n = ['직장인구(공통데이터).csv','상주인구(공통데이터).csv','생활인구(공통데이터).csv']  # 파일이름
f = '/content/drive/MyDrive/단위프로젝트_1조/공통전처리'   # 폴더 경로

for df, filename in zip(dfs, n):   #  dfs 와 n을  df , filename 대입 
    sgg_df = df[df['시군구명'] == sgg]  # 시군구명에 구명 넣고
    new = os.path.join(f, filename.replace('.csv', f'_{sgg}.csv'))  #  파일 이름에 해당 구를 추가하고 폴더 경로와 합해서 새로운 파일 경로를 생성하고 new 대입
    sgg_df.to_csv(new, index=False) #  /content/drive/MyDrive/상권분석/(상권-OOOO)_주소추가_마포구.csv 로 저장

## 2.2. 분기별로 저장(4분기만)

In [17]:
ydf1 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/직장인구(공통데이터)_영등포구.csv')
ydf2 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/상주인구(공통데이터)_영등포구.csv')
ydf3 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/생활인구(공통데이터)_영등포구.csv')

In [18]:
bg = 4                    
dfs = [ydf1, ydf2, ydf3]
n = ['직장인구(공통데이터)_영등포구.csv','상주인구(공통데이터)_영등포구.csv','생활인구(공통데이터)_영등포구.csv']
f = '/content/drive/MyDrive/단위프로젝트_1조/공통전처리'

for df, filename in zip(dfs, n):
    bg_df = df[df['기준_분기_코드'] == bg]
    new = os.path.join(f, filename.replace('.csv', f'_{bg}.csv'))
    bg_df.to_csv(new, index=False)

# 3. 데이터 확인 및 전처리

## 3.1. 파일 용량 줄이기

In [19]:
ydf1 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/직장인구(공통데이터)_영등포구_4.csv')
ydf2 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/상주인구(공통데이터)_영등포구_4.csv')
ydf3 = pd.read_csv('/content/drive/MyDrive/단위프로젝트_1조/공통전처리/생활인구(공통데이터)_영등포구_4.csv')

In [20]:
# 파일 용량 줄이기 위해 데이터타입 int32이하로 변경

dfs = [ydf1,ydf2, ydf3]

for df in dfs:
    for col in df.select_dtypes(include='int64').columns:
        if not df[col].isna().any():
            try:
                if df[col].max() <= 255:
                    df[col] = df[col].astype('Int8')
                elif df[col].max() <= 65535:
                    df[col] = df[col].astype('Int16')
                elif df[col].max() <= 4294967295:
                    df[col] = df[col].astype('Int32')
            except TypeError:
                pass

## 3.2. 컬럼명 통일

In [34]:
ydf1.rename(columns={'기준_년월_코드':'기준_년_코드'}, inplace=True)
ydf1.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수', '연령대_10_직장_인구_수',
       '연령대_20_직장_인구_수', '연령대_30_직장_인구_수', '연령대_40_직장_인구_수', '연령대_50_직장_인구_수',
       '연령대_60_이상_직장_인구_수', '남성연령대_10_직장_인구_수', '남성연령대_20_직장_인구_수',
       '남성연령대_30_직장_인구_수', '남성연령대_40_직장_인구_수', '남성연령대_50_직장_인구_수',
       '남성연령대_60_이상_직장_인구_수', '여성연령대_10_직장_인구_수', '여성연령대_20_직장_인구_수',
       '여성연령대_30_직장_인구_수', '여성연령대_40_직장_인구_수', '여성연령대_50_직장_인구_수',
       '여성연령대_60_이상_직장_인구_수', '행정동_코드', '시군구명', '읍면동명', 'lon', 'lat'],
      dtype='object')

In [35]:
# 2022년 4분기만 남기기 위해 2022로 필터링
ydf1_2022 = ydf1[ydf1['기준_년_코드']==2022]
ydf2_2022 = ydf2[ydf2['기준_년_코드']==2022]
ydf3_2022 = ydf3[ydf3['기준_년_코드']==2022]

In [37]:
ydf1_ch = ydf1_2022[['읍면동명','상권_코드_명','총_직장_인구_수']]
ydf1_ch

,읍면동명,상권_코드_명,총_직장_인구_수
0,신길제1동,대신시장,12
6,영등포동,영등포시장기계공구상가,158
12,신길제3동,신동시장,2
18,영등포동,영등포전통시장,45
24,영등포동,영신상가(제일상가),9
...,...,...,...
516,양평제2동,선유도역 6번,1792
522,문래동,문래예술촌(문래창작촌),4221
528,문래동,문래소공원,296
534,양평제1동,양평역 2번,1501


In [43]:
ydf1_ch = ydf1_ch.pivot_table(values='총_직장_인구_수', index='읍면동명', aggfunc = 'sum').sort_values('총_직장_인구_수', ascending = False)
ydf1_ch

,총_직장_인구_수
읍면동명,
여의동,108022
문래동,12755
영등포동,12078
당산제1동,9922
당산제2동,6852
양평제1동,6848
대림제3동,5800
양평제2동,5703
도림동,3895


## 3.3. 여의동만 저장

In [49]:
ydf2_2022.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '총 상주인구 수',
       '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수',
       '연령대 30 상주인구 수', '연령대 40 상주인구 수', '연령대 50 상주인구 수', '연령대 60 이상 상주인구 수',
       '남성연령대 10 상주인구 수', '남성연령대 20 상주인구 수', '남성연령대 30 상주인구 수',
       '남성연령대 40 상주인구 수', '남성연령대 50 상주인구 수', '남성연령대 60 이상 상주인구 수',
       '여성연령대 10 상주인구 수', '여성연령대 20 상주인구 수', '여성연령대 30 상주인구 수',
       '여성연령대 40 상주인구 수', '여성연령대 50 상주인구 수', '여성연령대 60 이상 상주인구 수', '총 가구 수',
       '아파트 가구 수', '비 아파트 가구 수', '상권 코드 명', '행정동_코드', '시군구명', '읍면동명', 'lon',
       'lat'],
      dtype='object')

In [50]:
ydf1_ch = ydf1_2022[['읍면동명','상권_코드_명','총_직장_인구_수']]
ydf2_ch = ydf2_2022[['읍면동명','상권 코드 명','총 상주인구 수']]
ydf3_ch = ydf3_2022[['읍면동명','상권_코드_명','총_생활인구_수']]

In [57]:
ydf1_yud = ydf1_ch[ydf1_ch['읍면동명']=='여의동']
yudjj = ydf1_yud.sort_values(by='총_직장_인구_수',ascending=False)
yudjj

,읍면동명,상권_코드_명,총_직장_인구_수
102,여의동,여의도역(여의도),69255
108,여의동,국회의사당역(국회의사당),31135
96,여의동,63빌딩,7632


In [65]:
jj = yudjj['총_직장_인구_수'].sum()

In [59]:
ydf2_yud = ydf2_ch[ydf2_ch['읍면동명']=='여의동']
yudsj = ydf2_yud.sort_values(by='총 상주인구 수',ascending=False)
yudsj

,읍면동명,상권 코드 명,총 상주인구 수
102,여의동,여의도역(여의도),2805
108,여의동,국회의사당역(국회의사당),2445
96,여의동,63빌딩,312


In [66]:
sj = yudsj['총 상주인구 수'].sum()

In [60]:
ydf3_yud = ydf3_ch[ydf3_ch['읍면동명']=='여의동']
yudsh = ydf3_yud.sort_values(by='총_생활인구_수',ascending=False)
yudsh

,읍면동명,상권_코드_명,총_생활인구_수
132,여의동,여의도역(여의도),1696105
138,여의동,국회의사당역(국회의사당),483456
126,여의동,63빌딩,61943


In [67]:
sh = yudsh['총_생활인구_수'].sum()

In [68]:
jj/(jj+sj+sh)

0.045867500492550596

## 3.4. 영등포동만 저장

In [69]:
ydf1_ydp = ydf1_ch[ydf1_ch['읍면동명']=='영등포동']
ydpjj = ydf1_ydp.sort_values(by='총_직장_인구_수',ascending=False)
ydpjj

,읍면동명,상권_코드_명,총_직장_인구_수
114,영등포동,한강성심병원,5045
120,영등포동,영등포역(영등포),3856
126,영등포동,영중로,2015
378,영등포동,영등포시장역 1번,292
282,영등포동,영등포동성당,227
6,영등포동,영등포시장기계공구상가,158
384,영등포동,영등포시장역 4번,127
246,영등포동,신길역 3번,102
306,영등포동,한림대학교한강성심병원,87
252,영등포동,서울시립영등포장애인복지관,76


In [76]:
jj = ydpjj['총_직장_인구_수'].sum()
jj

12078

In [71]:
ydf2_ydp = ydf2_ch[ydf2_ch['읍면동명']=='영등포동']
ydpsj = ydf2_ydp.sort_values(by='총 상주인구 수',ascending=False)
ydpsj

,읍면동명,상권 코드 명,총 상주인구 수
120,영등포동,영등포역(영등포),4753
114,영등포동,한강성심병원,2439
240,영등포동,신길역 3번,2234
126,영등포동,영중로,1588
306,영등포동,한림대학교한강성심병원,1264
18,영등포동,영등포전통시장,1053
282,영등포동,영등포동성당,838
384,영등포동,영등포시장역 1번,772
390,영등포동,영등포시장역 4번,763
6,영등포동,영등포시장기계공구상가,591


In [77]:
sj = ydpsj['총 상주인구 수'].sum()
sj

16893

In [73]:
ydf3_ydp = ydf3_ch[ydf3_ch['읍면동명']=='영등포동']
ydpsh = ydf3_ydp.sort_values(by='총_생활인구_수',ascending=False)
ydpsh

,읍면동명,상권_코드_명,총_생활인구_수
150,영등포동,영등포역(영등포),3203369
156,영등포동,영중로,1307835
144,영등포동,한강성심병원,451389
24,영등포동,영등포전통시장,404351
276,영등포동,신길역 3번,401255
420,영등포동,영등포시장역 1번,272986
12,영등포동,영등포시장기계공구상가,188364
426,영등포동,영등포시장역 4번,154567
342,영등포동,한림대학교한강성심병원,93138
390,영등포동,영등포시장역 3번,88622


In [78]:
sh = ydpsh['총_생활인구_수'].sum()
sh

6930641

In [79]:
sh/(jj+sj+sh)

0.9958372679396496